In [1]:
import pickle
pickle_in = open("SantaClaraLandFireFuelLongLat.pickle","rb")
fuel_dict = pickle.load(pickle_in)
#duration of firebrand 2-3 minutes according to https://www.fs.fed.us/psw/publications/weise/psw_2010_weise(koo)001.pdf

In [2]:
fuel_dict_keys = list(fuel_dict.keys())

In [3]:
fuel_dict_keys.sort()

In [4]:
fuel_dict_keys

['36.73225442280849, -121.93031545897136',
 '36.7323265829097, -121.92998892847145',
 '36.73239874212214, -121.9296623973436',
 '36.73247090044582, -121.92933586558787',
 '36.732512291305525, -121.93040683373346',
 '36.73254305788074, -121.92900933320422',
 '36.73258445164492, -121.93008030219941',
 '36.73261521442692, -121.9286828001927',
 '36.73265661109557, -121.9297537700375',
 '36.73268737008431, -121.92835626655327',
 '36.73272876965743, -121.92942723724765',
 '36.73275952485291, -121.92802973228596',
 '36.73277015969839, -121.93049820912346',
 '36.73280092733054, -121.92910070382989',
 '36.732831678732786, -121.92770319739077',
 '36.73284232027596, -121.93017167655535',
 '36.732873084114885, -121.92877416978425',
 '36.73290383172385, -121.92737666186774',
 '36.73291447996478, -121.9298451433593',
 '36.73294524001049, -121.9284476351107',
 '36.732975983826115, -121.9270501257168',
 '36.73298663876478, -121.92951860953534',
 '36.733017395017235, -121.92812109980926',
 '36.73302802

In [5]:
pickle_in = open("SantaClaraStructureLongLatUSDA.pickle","rb")
structure_dict = pickle.load(pickle_in)

In [6]:
structure_dict_keys = list(structure_dict.keys())

In [7]:
structure_dict_keys.sort()

In [11]:
# import cfo
# import os
# forest = cfo.api()
# wind_speed_ids = forest.search(metric="WindSpeed")
# os.chdir("wind_speeds")
# for wind_speed_id in wind_speed_ids:
#     forest.download(wind_speed_id)
# os.chdir("..")

[]


In [26]:
os.listdir('wind_speeds')

['California-Weather-WindSpeed-2022-0610-03000m.tif',
 'California-Weather-WindSpeed-2021-0801-03000m.tif',
 'California-Weather-WindSpeed-2021-0909-03000m.tif',
 'California-Weather-WindSpeed-2021-0810-03000m.tif',
 'California-Weather-WindSpeed-2021-0827-03000m.tif',
 'California-Weather-WindSpeed-2021-0904-03000m.tif',
 'California-Weather-WindSpeed-2021-0809-03000m.tif',
 'California-Weather-WindSpeed-2021-0901-03000m.tif',
 'California-Weather-WindSpeed-2021-0822-03000m.tif',
 'California-Weather-WindSpeed-2021-0815-03000m.tif',
 'California-Weather-WindSpeed-2021-0804-03000m.tif',
 'California-Weather-WindSpeed-2021-0910-03000m.tif',
 'California-Weather-WindSpeed-2021-0818-03000m.tif',
 'California-Weather-WindSpeed-2022-0609-03000m.tif',
 'California-Weather-WindSpeed-2021-0730-03000m.tif',
 'California-Weather-WindSpeed-2021-0828-03000m.tif',
 'California-Weather-WindSpeed-2022-0612-03000m.tif',
 'California-Weather-WindSpeed-2021-0803-03000m.tif',
 'California-Weather-WindSpe

In [28]:
import numpy as np
import json
import rasterio
from rasterio.plot import show

wind_speed_tiffs = []
id_list = os.listdir("wind_speeds")
for wind_speed_id in id_list:
    fp = "wind_speeds/" + wind_speed_id
    img = rasterio.open(fp)
    affine = img.transform
    imarray = img.read(1)
    wind_speed_tiffs.append(imarray)
    
count = len(wind_speed_tiffs)

wind_dict = {}
i = 0
while i < len(imarray):
    j = 0
    epsg_coords = affine * (j, i)
    latitude = epsg_coords[1]
    while j < len(imarray[0]):
        epsg_coords = affine * (j, i)
        longitude = epsg_coords[0]
        total_wind_speed = 0
        for tiff in wind_speed_tiffs:
            total_wind_speed += tiff[i][j]
        wind_dict[str(latitude) + ", " + str(longitude)] = total_wind_speed / count
        j += 1
    i += 1
    if i % (len(imarray) // 100) == 0:
        print(str(int((i+1) / len(imarray) * 100)) + "%")

<open DatasetReader name='wind_speeds/California-Weather-WindSpeed-2022-0610-03000m.tif' mode='r'>
<open DatasetReader name='wind_speeds/California-Weather-WindSpeed-2021-0801-03000m.tif' mode='r'>
<open DatasetReader name='wind_speeds/California-Weather-WindSpeed-2021-0909-03000m.tif' mode='r'>
<open DatasetReader name='wind_speeds/California-Weather-WindSpeed-2021-0810-03000m.tif' mode='r'>
<open DatasetReader name='wind_speeds/California-Weather-WindSpeed-2021-0827-03000m.tif' mode='r'>
<open DatasetReader name='wind_speeds/California-Weather-WindSpeed-2021-0904-03000m.tif' mode='r'>
<open DatasetReader name='wind_speeds/California-Weather-WindSpeed-2021-0809-03000m.tif' mode='r'>
<open DatasetReader name='wind_speeds/California-Weather-WindSpeed-2021-0901-03000m.tif' mode='r'>
<open DatasetReader name='wind_speeds/California-Weather-WindSpeed-2021-0822-03000m.tif' mode='r'>
<open DatasetReader name='wind_speeds/California-Weather-WindSpeed-2021-0815-03000m.tif' mode='r'>
<open Data

In [29]:
wind_dict_keys = list(wind_dict.keys())

In [30]:
wind_dict_keys.sort()

In [31]:
from scipy import spatial
wind_tree_vals = []
wind_dict_keys_len = len(wind_dict_keys)
for i, key in enumerate(wind_dict_keys):
    key_lat = float(key.split(", ")[0])
    key_long = float(key.split(", ")[1])
    wind_tree_vals.append((key_lat, key_long))
    if i % (wind_dict_keys_len // 100) == 0:
        print(str(int((i + 1) / (wind_dict_keys_len) * 100)) + "%")
wind_tree = spatial.KDTree(wind_tree_vals)

0%
1%
2%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%


In [32]:
from scipy import spatial
fuel_tree_vals = []
fuel_dict_keys_len = len(fuel_dict_keys)
for i, key in enumerate(fuel_dict_keys):
    key_lat = float(key.split(", ")[0])
    key_long = float(key.split(", ")[1])
    fuel_tree_vals.append((key_lat, key_long))
    if i % (fuel_dict_keys_len // 100) == 0:
        print(str(int((i + 1) / (fuel_dict_keys_len) * 100)) + "%")
fuel_tree = spatial.KDTree(fuel_tree_vals)

0%
1%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%


In [33]:
structure_tree_vals = []
structure_dict_keys_len = len(structure_dict_keys)
for i, key in enumerate(structure_dict_keys):
    key_lat = float(key.split(", ")[0])
    key_long = float(key.split(", ")[1])
    structure_tree_vals.append((key_lat, key_long))
    if i % (structure_dict_keys_len // 100) == 0:
        print(str(int((i + 1) / (structure_dict_keys_len) * 100)) + "%")
structure_tree = spatial.KDTree(structure_tree_vals)

0%
1%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%


In [34]:
import math
def compute_distance(lat1, long1, lat2, long2):
    return math.sqrt((lat2-lat1) ** 2 + (long2-long1) ** 2)

In [35]:
structure_dict_keys

['36.732254422808445, -121.93031545897134',
 '36.732326582909664, -121.92998892847142',
 '36.73239874212213, -121.92966239734358',
 '36.73247090044584, -121.92933586558784',
 '36.7325122913055, -121.93040683373343',
 '36.73254305788072, -121.92900933320419',
 '36.7325844516449, -121.93008030219941',
 '36.73261521442686, -121.92868280019268',
 '36.73265661109553, -121.92975377003746',
 '36.7326873700843, -121.92835626655324',
 '36.73272876965737, -121.92942723724762',
 '36.73275952485293, -121.92802973228594',
 '36.73277015969834, -121.93049820912344',
 '36.73280092733047, -121.92910070382989',
 '36.73283167873276, -121.92770319739077',
 '36.73284232027589, -121.93017167655532',
 '36.73287308411484, -121.92877416978423',
 '36.73290383172384, -121.92737666186771',
 '36.73291447996474, -121.92984514335927',
 '36.73294524001044, -121.92844763511066',
 '36.73297598382611, -121.92705012571678',
 '36.73298663876477, -121.92951860953532',
 '36.73301739501717, -121.92812109980925',
 '36.7330280

In [36]:
first_coord = float(structure_dict_keys[0].split(", ")[False])
cur_coord = first_coord
i = 1
while cur_coord == first_coord:
    cur_coord = float(structure_dict_keys[i].split(", ")[False])
    i += 1
# structure_delta_lat, num_longs = (abs(cur_coord - first_coord), i-1)
structure_delta_long = abs(float(structure_dict_keys[0].split(", ")[1]) - float(structure_dict_keys[1].split(", ")[1]))
#The latitude is exact while the longitude is an estimation

In [37]:
delta_lat_30m = 0.00027027027

In [38]:
structure_delta_long

0.00032653049991893113

In [39]:
first_coord = float(fuel_dict_keys[0].split(", ")[False])
cur_coord = first_coord
i = 1
while cur_coord == first_coord:
    cur_coord = float(fuel_dict_keys[i].split(", ")[False])
    i += 1
# fuel_delta_lat, num_longs = (abs(cur_coord - first_coord), i-1)
fuel_delta_long = abs(float(fuel_dict_keys[0].split(", ")[1]) - float(fuel_dict_keys[1].split(", ")[1]))
#The latitude is exact while the longitude is an estimation

In [40]:
fuel_delta_long

0.00032653049991893113

In [41]:
def find_closest_key(tree, tree_vals, key):
    lat = key.split(", ")[0]
    long = key.split(", ")[1]
    closest_key_index = tree.query([(lat, long)])[1][0]
    closest_key_set = tree_vals[closest_key_index]
    return str(closest_key_set[0]) + ", " + str(closest_key_set[1])


In [ ]:
# rightmost,leftmost,topmost,bottommost = (-120.003773, -121.279784, 39.52692, 39.00516)
# delta_lat = 0.0017868493150684946
# delta_long = 0.0023413045871559838 #These are the deltas for the structure dictionary
# def find_delta_coord(l, isLong):
#     first_coord = float(l[0].split(", ")[isLong])
#     cur_coord = first_coord
#     i = 1
#     while cur_coord == first_coord:
#         cur_coord = float(l[i].split(", ")[isLong])
#         i += 1
#     return (abs(cur_coord - first_coord), i-1)

# def find_closest_coord(l, coord, isLong, delta_coord):
#     return find_closest_coord_helper(l, coord, delta_coord, isLong, 0, len(l) - 1)
# def find_closest_coord_helper(l, coord, delta_coord, isLong, beginning, end):
#     middle_ind = (beginning + end) // 2
#     middle_coord = float(l[middle_ind].split(", ")[isLong])
#     if abs(middle_coord - coord) <= (delta_coord) / 2:
#         return (middle_coord, middle_ind)
#     elif middle_coord > coord and coord > 0:
#         return find_closest_coord_helper(l, coord, delta_coord, isLong, beginning, middle_ind)
#     elif middle_coord < coord and coord > 0:
#         return find_closest_coord_helper(l, coord, delta_coord, isLong, middle_ind, end)
#     elif middle_coord > coord and coord < 0:
#         return find_closest_coord_helper(l, coord, delta_coord, isLong, middle_ind, end)
#     elif middle_coord < coord and coord < 0:
#         return find_closest_coord_helper(l, coord, delta_coord, isLong, beginning, middle_ind)
    
# def find_closest_key(l, key):
#     key_lat = float(key.split(", ")[0])
#     key_long = float(key.split(", ")[1])
#     delta_lat, num_lat = find_delta_coord(l, False)
#     delta_long, num_long = find_delta_coord(l, True)
#     try:
#         lat, index = find_closest_coord(l, key_lat, False, delta_lat)
#     except:
#         if key_lat < bottommost:
#             lat = bottommost
#         if key_lat > topmost:
#             lat = topmost
            
#     est_long = float(l[index].split(", ")[1])
#     while est_long < key_long and abs(key_long - est_long) > (delta_long) / 2:
#         index -= 1
#         est_long = float(l[index].split(", ")[1])
#     while est_long > key_long and abs(key_long - est_long) > (delta_long) / 2:
#         index += 1
#         est_long = float(l[index].split(", ")[1])
    
        
# #     try:
# #         long = find_closest_coord(l[:num_lat], key_long, True, delta_long)
# #     except:
# #         if key_long < leftmost:
# #             long = leftmost
# #         if key_long > rightmost:
# #             long = rightmost
#     return str(lat) + ", " + str(est_long)
    
    
    
    
    
#     '''for key in l:
#         key_lat = float(key.split(", ")[0])
#         key_long = float(key.split(", ")[1])
#         distance = compute_distance(key_lat, key_long, float(lat), float(long))
#         if shortest_distance is None or distance < shortest_distance:
#             shortest_distance = distance
#             shortest_distance_coordinates = key
#     return shortest_distance_coordinates'''
        
    

In [ ]:
from functools import partial

import shapely
import pyproj
from shapely import geometry
from shapely.geometry import Point
from shapely.ops import transform

def sum_radius(dictionary, tree, tree_vals, center_lat, center_long, delta_lat, delta_long, isLookingForStructures, default_deltas):
    
    
    total_count = 0
    if isLookingForStructures:
        wind_key = find_closest_key(wind_tree, wind_tree_vals, str(center_lat) + ", " + str(center_long))
        wind_speed = wind_dict[wind_key]#In KM
        wind_distance = wind_speed * 1000 * 2.5 / 60#2.5 is the measure in minutes of how long embers live
        wind_distance_in_degrees = wind_distance / 111139
        num_deltas = int(wind_distance_in_degrees / delta_lat) + 1
        ell = shapely.geometry.Point((center_long, center_lat)).buffer(wind_distance_in_degrees)
        ratio = wind_distance_in_degrees / num_deltas / delta_lat
    else:
        num_deltas = default_deltas
        ellipse = ((center_long, center_lat),(delta_long, delta_lat), 0)#the 0 is degrees rotated, however, code is not implemented

        # Let create a circle of radius 1 around center point:
        circ = shapely.geometry.Point((center_long, center_lat)).buffer(1)

        # Let create the ellipse along x and y:
        ell  = shapely.affinity.scale(circ, float(ellipse[1][0]) * num_deltas, float(ellipse[1][1]) * num_deltas)
    
        ratio = 1
    
    

    for i in range(num_deltas * 2 + 1):
        est_lat = center_lat - (i - 1 * num_deltas) * delta_lat
        for j in range(num_deltas * 2 + 1):
            est_long = center_long - (j - 1 * num_deltas) * delta_long
            try:
                closest_key = find_closest_key(tree, tree_vals, str(est_lat) + ", " + str(est_long))
                if ell.contains(Point(float(closest_key.split(", ")[1]), float(closest_key.split(", ")[0]))):
                    total_count += (dictionary[closest_key] if dictionary[closest_key] >= 0 else 0)
            except:
                pass
    if num_deltas == 0:
        try:
            closest_key = find_closest_key(tree, tree_vals, str(est_lat) + ", " + str(est_long))
            return dictionary[closest_key] if dictionary[closest_key] >= 0 else 0
        except:
            return 0
    return total_count * ratio

def Sort(sub_li, index):
  
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    return(sorted(sub_li, key = lambda x: x[index]))    
  


In [ ]:
fuel_tree.query([(37.5, -121.5)])[1][0]

In [ ]:
fuel_tree_vals[5860686]

In [ ]:
fuel_dict[str(fuel_tree_vals[7731291][0]) + ", " + str(fuel_tree_vals[7731291][1])]

In [ ]:
sum_radius(fuel_dict, fuel_tree, fuel_tree_vals, 39.26246414075903, -121.01279809828802, fuel_delta_lat, fuel_delta_long, False, 5)

In [ ]:
sum_radius(structure_dict, structure_tree, structure_tree_vals, 37.20008227713145, -121.69999527653717, structure_delta_lat, structure_delta_long, True, 0)

In [ ]:
# high_fuel_keys = []
# for key in fuel_dict_keys:
#     if fuel_dict[key] == 202:
#         high_fuel_keys.append(key)

In [ ]:
# high_fuel_keys

In [ ]:
#Same process but only look through the original high_fuel_keys for potential high radii

In [ ]:
fuel_radii = []
structure_dict_keys_len = len(structure_dict_keys)
for i, key in enumerate(structure_dict_keys):
    try:
        lat = key.split(", ")[0]
        long = key.split(", ")[1]
        closest_key_index = fuel_tree.query([(lat, long)])[1][0]
        closest_key_set = fuel_tree_vals[closest_key_index]
        fuel_key = str(closest_key_set[0]) + ", " + str(closest_key_set[1])
        total_fuel = sum_radius(fuel_dict, fuel_tree, fuel_tree_vals, float(fuel_key.split(", ")[0]), float(fuel_key.split(", ")[1]), fuel_delta_lat, fuel_delta_long, False, 3)
        fuel_radii.append((fuel_key, total_fuel))
    except:
        pass
    if i % (structure_dict_keys_len // 100) == 0:
        print(str(int((i + 1) / (structure_dict_keys_len) * 100)) + "%")

In [ ]:
fuel_radii = Sort(fuel_radii)
fuel_radii.reverse()

In [ ]:
fuel_radii

In [ ]:
import pickle
pickle_out = open("santa_clara_fuel_radii.pickle", "wb")
pickle.dump(fuel_radii, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("santa_clara_fuel_radii.pickle","rb")
fuel_radii = pickle.load(pickle_in)

In [ ]:
count = 0
high_fuel_keys = []
for key, val in fuel_radii:
    if val >= 4500:
        count += 1
        high_fuel_keys.append((key, val))

In [ ]:
high_fuel_keys

In [ ]:
count

In [ ]:
ranking = []
for i, (key, fuel) in enumerate(high_fuel_keys):
    structure_key = find_closest_key(structure_tree, structure_tree_vals, key)
    lat = float(structure_key.split(", ")[0])
    long = float(structure_key.split(", ")[1])
    total_structures = sum_radius(structure_dict, structure_tree, structure_tree_vals, lat, long, structure_delta_lat, structure_delta_long, True, 0)
    ranking.append((key, fuel, total_structures))
    if i % (len(high_fuel_keys) // 100) == 0:
        print(str(int((i + 1) / len(high_fuel_keys) * 100)) + "%")    

In [ ]:
test = Sort(ranking, 2)
test = Sort(test, 1)
test.reverse()
test

In [ ]:
test = Sort(ranking, 2)
test.reverse()
test

In [ ]:
# ranking = Sort2(ranking)
ranking.reverse()
ranking

In [ ]:
high_struct = []
for rank in ranking:
    if rank[2] >= 700_000:
        high_struct.append(rank)

In [ ]:
high_struct = Sort3(high_struct)
high_struct.reverse()
high_struct

In [ ]:
test_fuel_key = find_closest_key(fuel_tree, fuel_tree_vals, "37.21937060704488, -121.89032606590727")
fuel_dict[test_fuel_key]

In [ ]:
high_fuel_keys

In [ ]:
radii = [1, 3, 5, 8, 13, 21, 34, 55]
# radii = [1, 3, 5, 8, 13]

In [ ]:
#Take the radius of 5 of highest fuel. Take the top ones of those and do the sum structures.
#39.255784, -121.008057   # Very interesting spot to look at

In [ ]:
radii_ranking = []
import time
print(time.time())
for radius in radii:
    ranking = []
    for key in high_fuel_keys:
        structure_key = find_closest_key(structure_tree, structure_tree_vals, key)
        lat = float(structure_key.split(", ")[0])
        long = float(structure_key.split(", ")[1])
        total_structures = sum_radius(structure_dict, structure_tree, structure_tree_vals, lat, long, structure_delta_lat, structure_delta_long, radius)
        ranking.append((key, total_structures))
    ranking = Sort(ranking)
    ranking.reverse()
    radii_ranking.append((ranking, radius))
    print(radius)
    print(time.time())
    

In [ ]:
test_key = find_closest_key(structure_tree, structure_tree_vals, '39.3, -121.089391')

In [ ]:
test_key

In [ ]:
'''
Gather new coordinates--Done
Gather Data -- Tonight
Look into USDA -- 
Look into wind -- Sources say embers can travel up to 40 km
'''



In [ ]:
import pickle
pickle_out = open("ranking.pickle", "wb")
pickle.dump(ranking, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("ranking.pickle","rb")
old_ranking = pickle.load(pickle_in)

In [ ]:
old_ranking

In [ ]:
import pickle
pickle_in = open("radii_ranking.pickle","rb")
radii_ranking = pickle.load(pickle_in)

In [ ]:
radii_ranking[3][0]

In [ ]:
radii_ranking[4][0]

In [ ]:
"""
Old Plan- Go through each of the keys for the structure database and record the fuel at that location. 
Then, look for all the structures in the ellipse radius of the key and sum them. Finally add the product 
of the total structures and fuel to the product dictionary
"""
# 39.356104, -120.194721 Interesting LatLong to Look at

In [ ]:
product_dict = {}

In [ ]:
import time
index = 1
end = len(structure_dict_keys)
print(time.time())
for structure_key in structure_dict_keys:
    
    lat = float(structure_key.split(", ")[0])
    long = float(structure_key.split(", ")[1])
    if lat > bottommost and lat < topmost and long < rightmost and long > leftmost:
        try:
            fuel_key = find_closest_key(fuel_dict_keys, structure_key)
            fuel_value = fuel_dict[fuel_key]
        except:
            print(structure_key)
            fuel_value = 0
        
        total_structures = sum_radius()

        product_dict[structure_key] = total_structures * fuel_value
    if index % (end // 100) == 0:
        print(str(index / (end // 100)) + "%")
        print(time.time())
    index += 1
    
    
            
    

In [ ]:
product_dict_values = list(product_dict.values())
product_dict_values = Sort(product_dict_values)
product_dict_values.reverse()

In [ ]:
max(product_dict.values())

In [ ]:
for structure_key in structure_dict_keys:
    if structure_key not in product_dict_keys:
        product_dict[structure_key] = 0
        print(structure_key)

In [ ]:
import pickle
pickle_out = open("product_dict.pickle", "wb")
pickle.dump(product_dict, pickle_out)
pickle_out.close()

In [ ]:
products = []
latitude = []
longitude = []

for product_key in product_dict_keys:
    products.append(product_dict[product_key])
    latitude.append(product_key.split(", ")[0])
    longitude.append(product_key.split(", ")[1])

In [ ]:
import pandas as pd

# initialise data of lists.
data = {'Products': products,
		'Latitude':latitude,
        'Longitude':longitude}

# Creates pandas DataFrame.
df = pd.DataFrame(data)

# print the data
df


In [ ]:
import pickle
pickle_out = open("product_dataframe.pickle", "wb")
pickle.dump(df, pickle_out)
pickle_out.close()

Everything Below Here is Testing

In [ ]:
res = forest.search(geography="MendocinoCounty", metric="CanopyCover")

In [ ]:
import cfo
forest = cfo.api()
wind_ids = forest.search(metric="WindSpeed")

In [ ]:
wind_ids

In [ ]:
forest.download(wind_ids[0])

In [ ]:
import numpy as np
import PIL
from PIL import Image
im = Image.open('Nevada/NevadaCounty-Vegetation-SurfaceFuels-2020-Summer-00010m.tif')
imarray = np.array(im)

In [ ]:
imarray.shape

In [ ]:
imarray[2127][-1]

In [ ]:
arr = []
for j, row in enumerate(imarray):
    if row[5484] >= 180 and row[5484] > 0:
        arr.append((row[5484], j))

In [ ]:
arr = []
for j, val in enumerate(imarray[0]):
    if val >= 180:
        arr.append((val, j))

In [ ]:
arr

In [ ]:
%matplotlib auto

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(imarray, cmap='viridis', vmin=0, vmax=200)
plt.colorbar()
plt.title("Nevada County Surface Fuels (Spread Rate)")
# plt.savefig('NevadaMaps/NevadaCounty-SurfaceFuels2.png')
plt.show()

In [ ]:
from ipyleaflet import Map, WMSLayer, LayersControl, basemaps
wms = WMSLayer(
    url='https://maps.salo.ai/geoserver/cfo/wms?p0=0.0&p2=1.44&p25=18.0&p30=21.599999999999998&p50=36.0&p60=43.199999999999996&p75=54.0&p90=64.8&p98=70.56&p100=72.0',
    layers="cfo:MendocinoCounty-Vegetation-CanopyHeight-2020-Fall-00010m",
    name="Mendocino Canopy Height",
    styles="vegetation",
    format="image/png8",
    transparent=True,
    attribution="Forest Observatory © <a href=https://salo.ai'>Salo Sciences</a>",
)
m = Map(basemap=basemaps.Stamen.Terrain, center=(39.39,-123.33), zoom=10)
m.add_layer(wms)
control = LayersControl(position='topright')
m.add_control(control)
m

In [ ]:
from PIL import Image
im = Image.open('Nevada/NevadaCounty-Vegetation-SurfaceFuels-2020-Summer-00010m.tif')
im.show()


In [ ]:
import rasterio
from rasterio.plot import show
fp = 'Nevada/NevadaCounty-Vegetation-SurfaceFuels-2020-Summer-00010m.tif'
img = rasterio.open(fp)
show(img)

In [ ]:
img.transform

In [ ]:
band1 = img.read(1)

In [ ]:
band1.shape

In [ ]:
img.count, img.height, img.width

In [ ]:
print(img.crs)#salo is 32610

In [ ]:
from pyproj import Transformer
transformer = Transformer.from_crs("epsg:3857", "epsg:4326")
transformer.transform(-11705274.6374, 4826473.6922)

In [ ]:
import rasterio
import numpy as np
from affine import Affine
from pyproj import Proj, transform

fname = fp

# Read raster
with rasterio.open(fname) as r:
    T0 = r.transform  # upper-left pixel corner affine transform
    p1 = Proj(r.crs)
    A = r.read()  # pixel values

# All rows and columns
cols, rows = np.meshgrid(np.arange(A.shape[2]), np.arange(A.shape[1]))

# Get affine transform for pixel centres
T1 = T0 * Affine.translation(0.5, 0.5)
# Function to convert pixel row/column index (from 0) to easting/northing at centre
rc2en = lambda r, c: (c, r) * T1

# All eastings and northings (there is probably a faster way to do this)
eastings, northings = np.vectorize(rc2en, otypes=[float, float])(rows, cols)

# Project all longitudes, latitudes
p2 = Proj(proj='latlong',datum='WGS84')
longs, lats = transform(p1, p2, eastings, northings)

In [ ]:
import rasterio
from rasterio.plot import show
fp = 'California-Weather-WindSpeed-2021-0715-03000m.tif'
img = rasterio.open(fp)
show(img)

In [ ]:
print(img.crs)

In [ ]:
affine = img.transform
affine

In [ ]:
imarray = img.read(1)

In [ ]:
origin = affine * (0, 0)
end = affine * (len(imarray[0]), len(imarray))
origin, end

In [ ]:
img.bounds

In [ ]:
ma = (-999, 0, 0)
mi = (999, 0, 0)
for i in range(len(imarray)):
    for j in range(len(imarray[0])):
        if imarray[i][j] > ma[0]:
            ma = (imarray[i][j], i, j)
        if imarray[i][j] < mi[0]:
            mi = (imarray[i][j], i, j)  

In [ ]:
ma,mi

In [ ]:
point = affine * (100, 57)
point